In [1]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git


Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 499 (delta 0), reused 1 (delta 0), pack-reused 495
Receiving objects: 100% (499/499), 3.07 MiB | 13.86 MiB/s, done.
Resolving deltas: 100% (302/302), done.


In [2]:
!pip install lmdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 4.8 MB/s eta 0:00:00


In [3]:
%cd deep-text-recognition-benchmark

/content/deep-text-recognition-benchmark


In [4]:
!gdown 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9

Downloading...
From: https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
To: /content/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth
100% 199M/199M [00:00<00:00, 242MB/s]


In [5]:
!CUDA_VISIBLE_DEVICES=0 python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
demo_image/demo_1.png    	available                	0.9999
demo_image/demo_2.jpg    	shakeshack               	0.9530
demo_image/demo_3.png    	london               

In [6]:
!pip3 install fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 975.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=ba202a6a073ac5754862e49c250594fb81947313101e73bea71a0c97b059ac6e
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [ ]:
!unzip /content/drive/MyDrive/t/plate_img-train.zip -d /content/deep-text-recognition-benchmark

In [ ]:
!unzip /content/drive/MyDrive/t/plate_img-validation.zip -d /content/deep-text-recognition-benchmark

In [ ]:
import xml.etree.ElementTree as ET
import os

def is_latin_char(character):
    return character.isalpha() and ord(character) < 128

def is_latin_numeral(character):
    return character.isdigit() and ord(character) < 128

dataclass = ['train','validation']
dictionary = {
        'B': 'ب',
        'P': 'پ',
        'T': 'ت',
        'Y': 'ث',
        'Z': 'ز',
        'X': 'ش',
        'E': 'ع',
        'F': 'ف',
        'K': 'ک',
        'G': 'گ',
        'D': 'D',
        'S': 'S',
        'J': 'ج',
        'W': 'د',
        'C': 'س',
        'U': 'ص',
        'R': 'ط',
        'Q': 'ق',
        'L': 'ل',
        'M': 'م',
        'N': 'ن',
        'V': 'و',
        'H': 'ه',
        'I': 'ی',
        '0': '۰',
        '1': '۱',
        '2': '۲',
        '3': '۳',
        '4': '۴',
        '5': '۵',
        '6': '۶',
        '7': '۷',
        '8': '۸',
        '9': '۹',

}
Move = {
        'A': 'الف',
        '@': 'ویلچر',
    }
for i in dataclass:
    output_file_path = f'gt_{i}.txt'
    f = open(output_file_path, 'w')
    for file in os.listdir(f'{i}/'):
        # print(files)
        # for file in files:
        error_file = ''
        if file.endswith('.xml'):
            error_file = file
            # Parse the XML file
            tree = ET.parse(f'{i}/{file}')
            root = tree.getroot()
            # Access elements in the XML tree
            filename = root.find('filename').text
            print(f'Filename: {file}')
            name = ''
            new_name=''
            # Iterate through object elements
            for obj in root.findall('object'):
                name += obj.find('name').text
            # print(name)
            for key in Move:
                if name.find(Move[key]) != -1:
                    new_name=name.replace(Move[key], key)
                    name = ''
                    name = new_name
            new_name=''
            for index, v in enumerate(name):
                if is_latin_char(v)==False and is_latin_numeral(v) == False:
                    for key in dictionary:
                        if dictionary[key] == v:
                            new_name=name.replace(name[index], key)
                            name = ''
                            name = new_name
            # try:
            for g in name:
                if is_latin_char(g)==False and is_latin_numeral(g) == False and g != '@':
                    name = name.replace(g, "")
                # print(is_latin_char(g), is_latin_numeral(g))
            filejpg = file.split('.')
            filesave = filejpg[0] + ".jpg"
            f.write(f'{filesave}	{name}\n')

In [10]:
!python3 create_lmdb_dataset.py --inputPath train/ --gtFile gt_train.txt --outputPath dataset/train/

Written 1000 / 19381
Written 2000 / 19381
Written 3000 / 19381
Written 4000 / 19381
Written 5000 / 19381
Written 6000 / 19381
Written 7000 / 19381
Written 8000 / 19381
Written 9000 / 19381
Written 10000 / 19381
Written 11000 / 19381
Written 12000 / 19381
Written 13000 / 19381
Written 14000 / 19381
Written 15000 / 19381
Written 16000 / 19381
Written 17000 / 19381
Written 18000 / 19381
Written 19000 / 19381
Created dataset with 19381 samples


In [11]:
!python3 create_lmdb_dataset.py --inputPath validation/ --gtFile gt_validation.txt --outputPath dataset/validation/

Written 1000 / 2608
Written 2000 / 2608
Created dataset with 2608 samples


In [12]:
!python3 train.py \
--train_data dataset/train --valid_data dataset/validation \
--select_data / --batch_ratio 1 --num_iter 6000 --batch_max_length 8 --valInterval 100 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: dataset/train
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    dataset/train	 dataset: /
sub-directory:	/.	 num samples: 19285
num total samples of /: 19285 x 1.0 (total_data_usage_ratio) = 19285
num samples of / per batch: 192 x 1.0 (batch_ratio) = 192
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slown

In [16]:
!CUDA_VISIBLE_DEVICES=0 python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder /content/deep-text-recognition-benchmark/test_image/ \
--saved_model /content/deep-text-recognition-benchmark/saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from /content/deep-text-recognition-benchmark/saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
/content/deep-text-recognition-benchmark/test_image/00005.jpg	95t38633                 	